# OpenAI 의 API 를 이용해서 수능 국어 문제 풀어보기

## API 사용에 필요한 정보 가져오기

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

open_ai_organization_id = os.getenv("OPEN_AI_ORGANIZATION_ID")
open_ai_project_id = os.getenv("OPEN_AI_PROJECT_ID")
open_ai_token = os.getenv("OPEN_AI_TOKEN")

## API Client 초기화

In [2]:
from openai import OpenAI

MODEL = "gpt-4o"

open_ai_client = OpenAI(
    organization=open_ai_organization_id,
    project=open_ai_project_id,
    api_key=open_ai_token,
)

In [3]:
# 사용 가능한 open ai 모델 확인
for model in open_ai_client.models.list():
    print(model.id)

gpt-4-turbo
gpt-4-turbo-2024-04-09
tts-1
tts-1-1106
chatgpt-4o-latest
dall-e-2
gpt-4-turbo-preview
gpt-4o-mini
gpt-4o-mini-2024-07-18
gpt-3.5-turbo-instruct
gpt-4-0125-preview
gpt-3.5-turbo-0125
gpt-4o-2024-08-06
gpt-3.5-turbo
babbage-002
davinci-002
gpt-4o-realtime-preview-2024-10-01
dall-e-3
gpt-4o-realtime-preview
gpt-4o-2024-05-13
tts-1-hd
gpt-4o
tts-1-hd-1106
gpt-4-1106-preview
text-embedding-ada-002
gpt-3.5-turbo-16k
text-embedding-3-small
text-embedding-3-large
whisper-1
gpt-3.5-turbo-1106
gpt-4-0613
gpt-4
gpt-3.5-turbo-instruct-0914


### prediction 함수 정의하기
- prediction 이라는 함수에서 OpenAI 의 API 를 호출하여 수능 국어 문제를 풀고 그 답을 반환하도록 합니다.

In [4]:
from openai.types.chat import ChatCompletionMessage
import re


def create_message(role: str, content: str) -> dict:
    return {"role": role, "content": [{"type": "text", "text": content}]}


def create_system_message(content: str) -> dict:
    return create_message("system", content)


def create_user_message(content: str) -> dict:
    return create_message("user", content)


def extract_and_format_answer(answer_message: ChatCompletionMessage) -> int:
    pattern = r"(\d+)번"
    match = re.search(pattern, answer_message.content)
    if match:
        return int(match.group(1))
    else:
        return -1


def prediction(paragraph: str, question: str, choices: list) -> int:
    choices_str = "\n".join([f"{i + 1}번 {choice}" for i, choice in enumerate(choices)])
    messages = [
        create_system_message(
            """넌 한국의 국어 교육을 전공한 사람으로, 학생을 가르치는 선생님이야.
이제 한국의 SAT 라고 불리는 대학 수학 능력 시험 중 국어 부문의 문제를 풀고, 학생에게 답을 알려주어야해.
문제의 형식은 지문이 주어지고, 오지선다에서 정답 1개를 선택하는,즉 객관식이야.
학생인 문제를 물어보면 답인 번호를 말해줘."""
        ),
        create_user_message(
            f"""지문을 읽고, 문제의 답의 번호를 말해줘. 오직 번호만 말해줘.

지문: 
{paragraph}
---
문제: 
{question}
---
선택지:
{choices_str}"""
        ),
    ]

    completion = open_ai_client.chat.completions.create(model=MODEL, messages=messages)

    return extract_and_format_answer(completion.choices[0].message)

## 수능 국어 문제 가져오기
- `problems.json` 파일은 글의 지문인 `paragraph` 와 객관식의 모임은 `problems` 를 가지는 json 객체를 가지는 리스트 입니다.
- 아래와 같은 구조로 되어있습니다.
    ```JSON
    [
        {
            "paragraph": "지문",
            "problems": [
                {
                    "question": "문제",
                    "choices": ["5개의 선택지가 나열되어있는 리스트", "..."],
                    "answer": 3, // 정답 번호
                    "score": 3 // 이 문제의 정답
                }, {
                    "question": "문제",
                    "choices": ["5개의 선택지가 나열되어있는 리스트", "..."],
                    "answer": 1, // 정답 번호
                    "score": 3 // 이 문제의 정답
                } // ...
            ]
        }
    // ...
    ]
    ```

In [5]:
import json

with open("problems.json", "r") as f:
    korean_problems = json.load(f)

## prediction 으로 수능 국어 문제 풀기

In [6]:
total_scores = 0
for korean_problem in korean_problems:
    paragraph = korean_problem["paragraph"]
    for problem in korean_problem["problems"]:
        question = problem["question"]
        choices = problem["choices"]
        answer = problem["answer"]
        score = problem["score"]

        pred = prediction(paragraph, question, choices)
        if answer == pred:
            total_scores += score

print(f"{MODEL} 의 수능 국어 문제 점수는 {total_scores} 입니다.")

gpt-4o 의 수능 국어 문제 점수는 75 입니다.
